# Практическое задание 1

# Ранжирование вопросов StackOverflow с помощью векторных представлений слов

## курс "Математические методы анализа текстов"


### ФИО: <впишите>

## Внимание! Эта версия задания для тех, кто посещает курс в МФТИ.

## Введение

В этом задании вы научитесь вычислять близость текстов и применить этот метод для поиска похожих вопросов на [StackOverflow](https://stackoverflow.com).

### Используемые библиотеки

В данном задании потребуются следующие библиотеки:
- [Gensim](https://radimrehurek.com/gensim/) — инструмент для решения различных задач NLP (тематическое моделирование, представление текстов, ...).
- [Numpy](http://www.numpy.org) — библиотека для научных вычислений.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — библилиотека с многими реализованными алгоритмами машинного обучения для анализа данных.
- [Nltk](http://www.nltk.org) — инструмент для работы с естественными языками.
- [Pytorch](https://pytorch.org/) — инструмент для обучения нейросетей.


### Данные

Данные лежат в архиве `StackOverflowData.zip`, который состоит из:
- `train.tsv` - обучающая выборка. В каждой строке через табуляцию записаны дублирующие друг друга предложения;
- `test.tsv` - тестовая выборка. В каждой строке через табуляцию записаны: *<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...*

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_)

#### Тесты

In [ ]:
from tests import TaskTests

task_tests = TaskTests.from_json(path='test_gt.json')

### Вектора слов

Для решения вам потребуются предобученная модель векторных представлений слов. Используйте [модель эмбеддингов](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit), которая была обучена с помощью пакета word2vec на данных Google News (100 миллиардов слов). Модель содержит 300-мерные вектора для 3 миллионов слов и фраз. Вы можете скачать их, запустив блок кода ниже.

In [ ]:
from download_utils import download_google_vectors


download_google_vectors(target_dir='.')

## Часть 1. Предобученные векторные представления слов (2 балла)

Скачайте предобученные вектора и загрузите их с помощью функции [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) библиотеки Gensim с параметром *binary=True*. Если суммарный размер векторов больше, чем доступная память, то вы можете загрузите только часть векторов, указав параметр *limit* (рекомендуемое значение: 500000).

In [ ]:
import gensim


wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000,
)

### Как пользоваться этими векторами?

Как только вы загрузите векторные представления слов в память, убедитесь, что имеете к ним доступ. Сначала вы можете проверить, содержится ли какое-то слово в загруженных эмбедингах:

    'word' in wv_embeddings

Затем, чтобы получить соответствующий вектор, вы можете использовать оператор доступа по ключу:

    wv_embeddings['word']

### Проверим, корректны ли векторные представления

Чтобы предотвратить возможные ошибки во время первого этапа, можно проверить, что загруженные вектора корректны. Для этого проверьте три пункта:
1. Используя метод `.most_similar(positive=..., negative=...)`, найти слово, похожее на `woman`, `king` и непохожее на `man`.
2. Используя метод `.doesnt_match(...)`, найти "белую ворону" в списке `['breakfast, 'dinner', 'lunch', 'cereal']`.
3. Используя метод `.most_similar_to_given(word, [...])`, найти наиболее похожее на `music` слово из списка `['water', 'sound', 'backpack', 'mouse']`.

Прокомментируйте полученные результаты: считаете ли вы их верными и почему.

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

### Ранжирование вопросов StackOverflow

Давайте посмотрим на данные, которые мы будем использовать в рамках задания. Выборка уже разбита на обучающую и тестовую. Все файлы используют табуляцию в качестве разделителя, но они имеют разный формат:

- *обучающая* выборка (train.tsv) содержит похожие друг на друга предложения в каждой строке;
- *тестовая* выборка (validation.tsv) содержит в каждой строке: *вопрос, похожий вопрос, отрицательный пример 1, отрицательный пример 2, ...*

Считайте тестовую (валидационную) выборку. Ответьте на следующие вопросы:
1. Сколько пар-дубликатов предоставлено в выборке?
2. Сколько в среднем на каждую пару предоставлено отрицательных примеров?

In [ ]:
import tqdm


def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

validation = read_corpus('data/validation.tsv')

In [ ]:
import numpy as np

###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

In [ ]:
task_tests.test_validation_corpus(
    num_samples,
    amount_of_negatives_per_sample
)

### Векторные представления текста

Чтобы перейти от отдельных слов к векторным представлениям вопросов, предлагается подсчитать **среднее** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектоора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.



In [ ]:
import numpy as np


class Embedder:
    
    def __init__(self, embeddings, dim):
        """
            embeddings: word2vec эмбеддинги
            dim: размерность word2vec эмбеддингов. Нужна для задания нулего вектора для пустых вопросов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        
    def __call__(self, text, normalize=False):
        """
            Принимает на вход текст и преобразует его в вектор.
            
            text: строка с вопросом
            normalize: при True нужно перед возвращением нормализовать вектор
            
            returns: вектор вопроса
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################

In [ ]:
embedder = Embedder(wv_embeddings, dim=300)

In [ ]:
task_tests.test_embedder(embedder)

Теперь у нас есть метод для создания векторного представления любого предложения. Оценим, как будет работать это решение.

### Оценка близости текстов

В качестве метрики схожести вопросов будем использовать косинусную близость.

В валидационном датасете для каждой пары вопросов-дубликатов у нас есть случайные отрицательные примеры. Для каждого триплета (вопрос, дубликат, отрицательные примеры) будет ранжировать с помощью нашей модели и косинусной близости дубликат и отрицательные примеры и смотреть на позицию дубликата.

#### Hits@K
Довольно простой и легко интерпретируемой метрикой будет количество корректных попаданий дубликата в top "выдачи" для какого-то *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)],$$
где $q_i$ - $i$-ый вопрос, $dup_i$ - его дубликат, $topK(q_i)$ - первые *K* элементов в ранжированном списке, который выдает наша модель.

#### Пример оценок

Пусть $N = 1$, вопрос $q_1$ это "Что такое python", а его дубликат $dup_1$ это "Что такое язык python". Пусть модель выдала следующий ранжированный список кандидатов:

1. *"Как узнать с++"*
2. *"Что такое язык python"*
3. *"Хочу учить Java"*
4. *"Не понимаю Tensorflow"*

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [dup_1 \in top1(q_1)] = 0$
- [K = 4] $\text{Hits@4} =  [dup_1 \in top4(q_1)] = 1$

#### Подсчет метрики Hits@k сразу для нескольких k

Чтобы посчитать метрику для нескольких k, не нужно повторно ранжировать нашей моделью вопросы для одного и того же сэмпла. Достаточно посчитать для сэмпла количество **сложных негативов** - отрицательных примеров, оказавшихся в выдаче выше, чем дубликат. Тогда
$$Hits@k = \begin{cases}
    1, & N < k \\
    0, & иначе
   \end{cases},$$
где **N** - количество сложных негативов.

Реализуйте подсчет Hits@k для произвольного набора значений k и заданной валидационной выборки, используя предложенный шаблон.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
        
    
class Scorer:
    
    def __init__(self, k, embedder):
        """
            k: список значений k, для которых нужно посчитать hits@k
            embedder: объект класса Embedder, умеющий преобразовать текст в вектор
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        
    def _get_hard_negatives(self, q, pos, negs):
        """
            q: текст вопроса
            pos: текст дубликата
            negs: список из текстов случайных вопросов
            
            result: количество сложных отрицательных примеров, оказавшихся выше положительного
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def __call__(self, samples, verbose=False):
        """
            samples: список из списков вида [q, pos, neg1, neg2, ...]. Наша валидационная выборка
            verbose: выводить progressbar подсчета метрики с помощью tqdm
            
            result: словарь вида {k: hits@k}
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################

In [ ]:
scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)

hits = scorer(validation, verbose=True)

In [ ]:
task_tests.test_scorer(hits)

### Предобработка текста

Как вы могли заметить, мы имеем дело с сырыми данными. Это означает, что там присутствует много опечаток, спецсимволов и заглавных букв. В нашем случае это все может привести к ситуации, когда для данных токенов нет предобученных векторов. Поэтому необходима предобработка.

Вам требуется:
- Перевести символы в нижний регистр;
- Заменить символы пунктуации и всевозможные плохие символы на пробелы;
- Удалить стопслова.
- Удалить слова с длиной меньше трех букв

Реализуйте предобработку текста, используя предложенный шаблон.

In [ ]:
import re
    
    
class TextPreprocessor:
    
    def __init__(self, characters, min_word_length=0, stopwords=None):
        """
            characters: список плохих символов
            min_word_length: минимальная допустимая длина для слов
            stopwords: множество фоновых слов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################

    def __call__(self, text):
        """
            text: текст для обработки
            
            returns: обработанный текст
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################

In [ ]:
task_tests.test_text_preprocessor(TextPreprocessor)

Множество фоновых слов можно взять из **nltk** с помощью `nltk.corpus.stopwords.words`, выкидываемые плохие символы и пунктуацию следует подобрать самостоятельно.

Обработайте текст и продемонстрируйте улучшение качества:

In [ ]:
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
    

###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

Одним из критериев получения полных баллов является значение **hits@500** $\geqslant 0.82$ до предобработки текста и $\geqslant 0.85$ после предобработки.

## Часть 2. Представления для неизвестных слов. (8 баллов)

Для того, что получить представления для неизвестного слова, воспользуемся следующим подходом:
    
1. Будем восстанавливать эмбеддинг неизвестного слова как сумму эмбеддингов буквенных триграмм. Например, слово where должно представляться суммой триграмм #wh, whe, her, ere, re#

2. В качестве обучающих данных будем использовать слова, для которых есть эмбеддинг в модели. Будем обучать эмбеддинги триграмм по выборке эмбеддингов с помощью функционала MSE:

$$L = \sum_{w \in W_{known}}\| f_{\theta}(w) - v_w \|^2 \to \min_{\theta}$$

где:

* $W_{known}$ — множество известных модели слов
* $f_{\theta}(w)$ — сумма эмбеддингов триграмм слова $w$
* $v_w$ — эмбеддинг слова $w$
* $\theta$ — веса эмбеддингов триграмм

### Создание триграммного токенизатора

Для начала, нам нужно:
1. Пройтись по известным в word2vec словам и составить множество триграмм, для которых будем обучать векторы
2. Составить маппинг из триграмм в индексы
3. Реализовать преобразование произвольного слова в список триграмм
4. Реализовать преобразование произвольного слова в список индексов триграмм

Для реализации всех этих пунктов предлагается использовать шаблон, приведенный ниже.

In [ ]:
class TrigramTokenizer:
    
    def __init__(self, words):
        """
            Формируем множество всевозможных триграмм, встречающихся в словах из words.
            Делаем маппинг триграмм в индексы.
            
            words: список слов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        
    @property
    def vocab_size(self):
        """
            returns: колчиество триграмм, для которых мы завели индекс.
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    @staticmethod
    def _get_trigrams(word):
        """
            word: слово
            
            returns: список триграмм для word
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        
    def __call__(self, word):
        """
            word: слово
            
            returns: список индексов триграмм для слова word, которые нашлись в маппинге
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################

In [ ]:
task_tests.test_trigram_tokenizer(TrigramTokenizer)

Для создания токенизатора используйте обработанный с помощью TextProcessor текст. 

**Важно:** в токенизатор нужно подавать только слова, известные word2vec'у.

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

### Создание датасета с w2v векторами и списками индексов триграмм

Мы будем обучать триграммную модель в нейросетевом фреймворке pytorch. Для этого нам нужно создать свой датасет.

Он должен:
1. Принимать список слов, word2vec и уже созданный триграммный токенизатор.
2. Выдавать пары вида (эмбеддинг для слова из word2vec, список индексов триграмм для этого слова)

Реализовать датасет нужно в шаблоне, приведенном ниже.

In [ ]:
from torch.utils.data import Dataset


class TrainTrigramDataset(Dataset):
    
    def __init__(self, vocab, w2v_embeddings, tri_tokenizer):
        """
            Формируем выборку для обучения триграммной модели.
            ЗАРАНЕЕ считаем маппинг в список индексов для всех известных в word2vec слов.
            
            vocab: список слов
            w2v_embeddings: no comments
            tri_tokenizer: токенизатор триграмм
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
                
    def __len__(self):
        """
            returns: возвращает количество слов, вошедших в маппинг (размер словаря)
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def __getitem__(self, idx):
        """
            returns: w2v эмбеддинг для idx-го слова в датасете, список соответствующих ему триграмм (тензоры)
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
ds = TrainTrigramDataset(w2v_vocab, wv_embeddings, tri_tokenizer)

In [ ]:
task_tests.test_dataset(ds, w2v_vocab, wv_embeddings, tri_tokenizer)

### Создание DataLoader'а и Collator'а

Нас интересуют в первую очередь четыре параметра при создании DataLoader:
1. Датасет. Реализует интерфейс массива - можно узнать длину и получить элемент с индексом, меньшим длины.
2. batch_size. Задает размера батча (количества сэмплов, идущих одновременно в модель).
3. shuffle. При shuffle == True каждую эпоху при итерировании по даталоадеру мы будем получать сэмплы в произвольном порядке.
4. collate_fn. Этот параметр позволяет задать кастомную логику "склеивания" сэмплов из датасета в батч.

В качестве модели мы будем использовать слой **torch.nn.EmbeddingBag**. Он принимает на вход список индексов и список сдвигов, начинающийся с нуля.

Нужно наш список списков индексов триграмм превратить в соответствующий формат, преобразовать векторы слов и два списка (индексов и сдвигов) в pytorch тензоры (torch.tensor).

Реализуйте следующую функцию:

In [ ]:
import torch
from torch.utils.data import DataLoader


def collate_fn(batch):
    """
        batch: список из элементов датасета, e.g. [ds[i] for i in [2, 3, 1, 15]]
        
        returns: w2v эмбеддинги, индексы триграмм, сдвиги для триграмм
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################

In [ ]:
task_tests.test_dataloader(ds, collate_fn, embedding_dim=300)

### Создание модели

При создании модели мы обычно наследуемся от **torch.nn.Module** и создаем нужные нам слои как атрибуты объекта нашего класса.

В данном случае предлагается для формирования эмбеддингов использовать **torch.nn.EmbeddingBag**.

Реализуйте предложенный шаблон:

In [ ]:
from torch import nn


class TrigramModel(nn.Module):
    
    def __init__(self, num_embeddings, embedding_dim):
        """
            num_embeddings: количество триграмм, для которых обучаются эмбеддинги
            embedding_dim: размерность эмбеддингов триграмм
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        
    @property
    def embedding_dim(self):
        """
            returns: размерность эмбеддингов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    @property
    def num_embeddings(self):
        """
            returns: количество эмбеддингов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def forward(self, trigrams, offsets):
        """
            trigrams: список индексов триграмм (тензор)
            offsets: список сдвигов (тензор)
            
            returns: эмбеддинги слов, составленные из триграмм
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
model = TrigramModel(tri_tokenizer.vocab_size, embedding_dim=wv_embeddings.vector_size)

In [ ]:
task_tests.test_trigram_model(model)

### Создание пайплайна обучения

Далее необходимо совместить все наработки в единый пайплайн обучения, добавив также критерий для оптимизации и оптимизатор. 

Предлагается:

1. В качестве оптимизатора использовать Adam (можно попробовать подобрать learning rate / weight decay)
2. В качестве критерия оптимизации взять nn.MSELoss (можно также закодить лосс самому)
3. Для даталоадера выбрать небольшой батч сайз (32, 64, 128, 256)
4. Десяти эпох должно быть достаточно для хорошего качества

Реализуйте предложенный шаблон.

In [ ]:
import time

    
class Trainer:
    
    def __init__(self, model, criterion, optimizer):
        """
            model: триграммная модель
            criterion: функционал ошибки, принимает на вход w2v эмбеддинги и триграммные эмбеддинги
            optimizer: оптимизатор для модели
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        
    def _train_step(self, dataloader):
        """
            Делаем один проход по даталоадеру, с бэкпропом
            
            dataloader: даталоадер с тренировочными данными
            
            returns: лосс
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def train(self, dataloader, n_epochs, verbose=False):
        """
            dataloader: тренировочный даталоадер
            n_epochs: количество эпох
            verbose: выводить лосс каждую эпоху или нет
            
            returns: список лоссов
        """
        start = time.time()
        losses = []
        for epoch in range(n_epochs):
            loss = self._train_step(dataloader)
            losses.append(loss)
            if verbose:
                print(f'epoch: {epoch + 1:>2}, loss: {loss:.4f}, time: {time.time() - start:.4f}')
        return losses


###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

### Получение векторов неизвестных слов. Инференс модели

Теперь, когда мы обучили модель, нам необходимо применить её для всех неизвестных слов, т.е. получить для них эмбеддинги.

Т.к. для этих слов у нас нет word2vec эмбеддингов, то dataset и collator для обучения не подходят для инференса. Необходимо реализовать датасет и коллатор для инференса по следующим шаблонам:

In [ ]:
class InferenceTrigramDataset:
    
    def __init__(self, vocab, tri_tokenizer):
        """
            Датасет с неизвестными словами
            
            vocab: список слов
            tri_tokenizer: триграммный токенизатор
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        
    def __len__(self):
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def __getitem__(self, idx):
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    
def inference_collate_fn(trigrams):
    """
        trigrams: список списков индексов триграмм
        
        returns: список индексов, список сдвигов триграмм
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################

Теперь у нас есть всё необходимое, чтобы осуществить инференс. Не забудь перед инференсом перевести модель в режим эвала (**model.eval**), а также использовать контекстный менеджер **torch.no_grad**.

После инференса сформируйте словарь из известных в word2vec слов и их эмбеддингов, затем дополните его эмбеддингами для неизвестных слов, полученными после инференса.

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

Используя **Scorer** и **Embedder**, получите новые значения метрик для валидации:

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

Одним из критериев получения полных баллов является значение метрики **hits@500** $\geqslant 0.89$.

## Дополнительная часть. Обучение векторных представлений для целевой задачи. (до 5 баллов)

Предполагается, что в этой части используются TextPreprocessor, Embedder, Scorer из предыдущих частей.

Для обучения на целевую задачу нам понадобится обучающая выборка. Считайте её с диска, предобработайте текст вопросов

In [ ]:
train = []
for questions in tqdm.tqdm(read_corpus('data/train.tsv')):
    train.append([text_preprocessor(text) for text in questions])

Необходимо создать **токенизатор для текста** - составить словарь и сделать маппинг из слов в индексы.

In [ ]:
class TextTokenizer:
    
    def __init__(self, vocab):
        """
            vocab: множество слов, встретившихся в обучающей выборке
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################

    @property
    def vocab_size(self):
        """
            returns: количество слов в словаре
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def __call__(self, text):
        """
            text: текст
            
            returns: список индексов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################

Составление словаря и токенизатора

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

Нам также понадобится **новый датасет для обучения**. Для применения метода NT-Exent нам не нужно "майнить негативы", поэтому датасет надо сформировать как массив из пар-дубликатов.

Так как данные в обучающей выборке содержат множества дубликатов (т.е. все дубликаты сгруппированы в списки), есть несколько способов сформировать итоговый датасет:
1. Оставить из каждого множества дубликатов какие-нибудь случайные два (или просто первые два вопроса)
2. Для первого вопроса в множестве взять все остальные как дубликаты (N вопросов-дубликатов - N-1 пара). Тогда мы увидим каждый вопрос хотя бы один раз при обучении
3. Составить всевозможные уникальные пары-дубликаты из этих множеств (т.е. первый вопрос и все остальные вопросы, второй вопрос и все остальные, кроме первого).

Каждый следующий способ, начиная с первого, раздувает выборку по размеру, но возможно дает прирост к качеству решения задачи.

Реализуйте выбранный вами подход, используя предолженный шаблон:

In [ ]:
class QuestionDuplicatesDataset(Dataset):
    
    def __init__(self, question_pairs, tokenizer):
        """
            question_pairs: список из пар вопросов-дубликатов
            tokenizer: объект класса TextTokenizer
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
            
    def __len__(self):
        """
            returns: количество пар-дубликатов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def __getitem__(self, idx):
        """
            returns: (вопрос, дубликат), idx-ю пару в датасете
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################

Также нужно подготовить **даталоадер** (а именно - коллатор для даталоадера) по аналогии со второй частью задания.

In [ ]:
def get_ids_and_offsets(questions):
    """
        questions: список из токенизированных вопросов
        
        returns: (ids, offsets), где ids - вытянутый список индексов слов в вопросах из батча, offsets - сдвиги
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################


def collate_fn(batch):
    """
        batch: список из пар токенизированных вопросов-дубликатов [(question, duplicate), ...]
        
        returns: (question_ids, question_offsets), (duplicate_ids, duplicate_offsets)
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################

Поделите выборку на трейн и валидацию, используя train_test_split, затем **создайте датасеты и даталоадеры** для обучения и валидации. Сколько пар-дубликатов получилось в датасете для обучения?

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

С помощью предложенного шаблона **задайте модель** для преобразования вопросов в векторы.

In [ ]:
from torch import nn


class DssmLikeModel(nn.Module):
    
    def __init__(self, num_embeddings, embedding_dim):
        """
            num_embeddings: количество слов, для которых обучаем эмбеддинги
            embedding_dim: размерность эмбеддинга
        """
        super().__init__()
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        
    def forward(self, ids, offsets):
        """
            ids: вытянутая посл-ть индексов слов вопросов, попавших в батч
            offsets: сдвиги для вопросов, попавших в батч
            
            returns: векторы вопросов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################

Создание модели

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

**Критерий оптимизации** для NTExentLoss выглядит как:

$$\mathcal{L}(Q, D) = -0.5 \log diag(softmax(QD^T / \alpha)) - 0.5 \log diag(softmax(DQ^T / \alpha)),$$

где:
* $Q \in \mathbb{R}^{b \times d}$ - эмбеддинги вопросов, 
* $D \in \mathbb{R}^{b \times d}$ - эмбеддинги соответствующих вопросам дубликатов,
* $b$ - количество пар (вопрос, дубликат), $d$ - размерность эмбеддингов, $\alpha$ - гиперпараметр лосса. 
* Softmax берется по рядам
* Матрицы $Q, D$ содержат нормированные эмбеддинги, т.е. считается именно косинус.

In [ ]:
class NTExentLoss(nn.Module):
    
    def __init__(self, alpha=1., eps=1e-8):
        """
            alpha: коэффициент, на который мы делим скоры перед софтмаксом
            eps: ||v|| = min(eps, ||v||)
        """
        super().__init__()
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        
    def _normalize(self, embeddings):
        """
            embeddings: матрица размера [batch_size, embedding_dim]
            
            returns: матрица такого же размера, но с нормироваными векторами
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def forward(self, embeddings, positives):
        """
            embeddings: матрица размера [batch_size, embedding_dim]
            positives: матрица такого же размера, с позитивами для векторов из матрицы embeddings
            
            returns: NT-Exent loss
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################

**Создайте пайплайн** для обучения и валидации, используя предложенный шаблон.

Залогируйте с помощью **torch.utils.tensorboard.SummaryWriter** две величины:
1. Лосс для каждого батча
2. Лосс на валидации для каждой эпохи

In [ ]:
import time
import shutil
import os
import torch

from torch.utils.tensorboard import SummaryWriter
    
    
class Trainer:
    
    def __init__(
            self, 
            model, 
            optimizer, 
            criterion, 
            logdir=None, 
            device=None
    ):
        """
            model: объект класса DssmModel
            optimizer: оптимизатор
            criterion: критерий оптимизации
            logdir: директория, в которую SummaryWriter должен писать логи
            device: девайс (cpu или cuda), на котором надо производить вычисления
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def _calculate_loss(self, batch):
        """
            batch: батч из индексов и сдвигов для вопросов и их дубликатов
            
            returns: посчитанный для батча лосс
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def _train_step(self, dataloader):
        """
            dataloader: даталоадер для обучения
            
            returns: лосс на датасете для обучения
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def _eval_step(self, dataloader):
        """
            dataloader: даталоадер для валидации
            
            returns: лосс на валидации
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def train(self, dataloaders, n_epochs, verbose=False):
        """
            dataloaders: словарь вида {'train': train_dataloader, 'eval': eval_dataloader}
            n_epochs: количество эпох обучения
            verbose: нужно ли выводить каждую эпоху информацию про лоссы
        """
        start = time.time()
        for epoch in range(n_epochs):
            train_loss = self._train_step(dataloaders['train'])
            
            eval_loss = self._eval_step(dataloaders['eval'])
            if self._writer is not None:
                self._writer.add_scalar('eval/loss', eval_loss, global_step=self._n_epoch)
                
            if verbose:
                print(
                    'epoch: {:>2}, train loss: {:.4f}, eval loss: {:.4f}, time: {:.4f}' \
                        .format(epoch + 1, train_loss, eval_loss, time.time() - start)
                )
                    
            self._n_epoch += 1

Предлагается использовать для оптимизации Адам и обучать модель 10-60 эпох.

Для этой части задания GPU даёт существенное ускорение при обучении, поэтому стоит по возможности делать обучение с большим batch size'ом и на GPU.

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

Осталось достать из модели обученные под задачу векторы слов, составить маппинг слов в векторы, создать **Embedder** и **Scorer** и провалидировать качество на нашей исходной валидации, которой мы пользовались в первых двух частях.

Чтобы достать из модели веса, можно использовать `model._embeddings.weight.cpu().detach().numpy()`

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

Одним из критериев получения полных баллов является значение метрики **hits@500** $\geqslant 0.98$.